**_Autor_**: Rubén del Mazo Rodríguez

# Implementación efectiva de redes neuronales
Una de las razones por las que los investigadores del aprendizaje profundo han sido capaces de escalar las redes neuronales, y desarrollar redes neuronales realmente grandes durante las últimas décadas, es debido a que las redes neuronales se pueden vectorizar. Se pueden implementar de forma muy eficiente utilizando multiplicaciones matriciales y resulta que el hardware de computación paralela, incluidas las GPU, pero también algunas funciones de CPU, son muy buenas haciendo multiplicaciones matriciales muy grandes. A continuación, veamos cómo funcionan estas implementaciones vectorizadas de redes neuronales. Sin estas ideas, es probable que el aprendizaje profundo no se acercara ni de lejos al éxito y la escala actuales.

## Tabla de contenidos
- [Aclaraciones sobre los arrays de numpy y sus dimensiones](#1)
- [Breve repaso de conceptos del álgebra](#2)
    - [Producto escalar](#2-1)
        - [np.dot()](#2-1-1)
    - [Multiplicación matricial](#2-2)
        - [np.matmul()](#2-2-1)
        - [a @ b](#2-2-2)
- [Vectorización en redes neuronales](#3)
    - [Cálculo de z](#3-0)
    - [Propragación hacia delante / _forward propagation_](#3-1)
    - [Implementación de las funciones de pérdida L1 y L2](#3-2)
        - [L1](#3-2-1)
        - [L2](#3-2-2)

In [1]:
# Importamos librerias

import numpy as np

<a name='1'></a>
### Aclaraciones sobre los arrays de numpy y sus dimensiones
NumPy es una biblioteca, creada en 2005, que se ha convertido en el estándar para el álgebra lineal en Python. Un hecho desafortunado sobre cómo se programa hoy en día es que hay bibliotecas que son utilizadas ampliamente para el aprendizaje profundo, como es TensorFlow, que utilizan NumPy y fueron creadas posteriormente, pero que son inconsistentes en cómo se representan los datos con respecto a NumPy. Por lo tanto, es importante conocer cómo se representa el álgebra lineal con NumPy a la hora de utilizarlo, tanto en solitario como con bibliotecas tan famosas como TensorFlow.

Veamos la representación de vectores y matrices en NumPy. Una matriz es un arreglo rectangular de magnitudes dispuestas en filas y columnas. Por convención, _n_ filas por _m_ columnas, _n x m_. Es importante mencionar que un vector se diferencia de una matriz en que una de sus dimensiones es adimensional (de magnitud uno), es decir, _n x 1_ o _1 x m_. Para crear matrices en dos dimensiones, hay que **utilizar doble corchete**: el primer corchete indica que se alberga la matriz, mientras que los siguientes corchetes albergan los números (o símbolos) que componen cada fila. Cada fila está separada por comas dentro del primer corchete. El número de columnas es el número de elementos de cada fila y este número tiene que ser el mismo en cada fila. En caso contrario, como es lógico, saltaría un error. Las dimensiones se pueden visualizar y utilizar en Python con la instrucción `.shape`.

En el caso de los vectores, si queremos que sea un vector 2D (y, por tanto, que siga siendo una matriz), hay que utilizar el doble corchete. Si solo utilizamos un corchete, estaríamos ante un **vector 1D**. Y he aquí la diferencia principal que hace importante esta aclaración sobre NumPy, porque bibliotecas como TensorFlow **trabajan con notación matricial** y no pueden trabajar con vectores 1D. TensorFlow fue diseñado para manejar conjuntos de datos muy grandes mediante la representación de los datos en matrices en lugar de matrices 1D, lo que permite a TensorFlow ser un poco más eficiente computacionalmente internamente.

In [2]:
# Ejemplo matriz 2x3
matriz_dos_por_tres = np.array([[1, 2, 3],
                                [4, 5, 6]])
matriz_dos_por_tres_equivalente = np.array([[1, 2, 3],[4, 5, 6]])

print(matriz_dos_por_tres)
print(matriz_dos_por_tres_equivalente)
print(matriz_dos_por_tres.shape)
print(matriz_dos_por_tres_equivalente.shape)

[[1 2 3]
 [4 5 6]]
[[1 2 3]
 [4 5 6]]
(2, 3)
(2, 3)


In [3]:
# Ejemplo matriz 4x2. En Python, puede haber int y double en la misma matriz, pero recordemos que los int se convertirán en doubles
# y será una matriz de doubles.
matriz_cuatro_por_dos = np.array([[1, 2], [-3.0, -4.0], [-0.5, -0.6], [6, 7]])

print(matriz_cuatro_por_dos)
print(matriz_cuatro_por_dos.shape)

[[ 1.   2. ]
 [-3.  -4. ]
 [-0.5 -0.6]
 [ 6.   7. ]]
(4, 2)


In [4]:
# Ejemplo vectores 1x2 y 2x1
vector_uno_por_dos = np.array([[100, 15]])
vector_dos_por_uno = np.array([[100], [15]])

print(vector_uno_por_dos)
print(vector_uno_por_dos.shape)
print(vector_dos_por_uno)
print(vector_dos_por_uno.shape)

[[100  15]]
(1, 2)
[[100]
 [ 15]]
(2, 1)


In [5]:
# Ejemplo vector 1D
vector_unidimensional = np.array([100, 15])

print(vector_unidimensional)
print(vector_unidimensional.shape)

[100  15]
(2,)


Como se puede observar en el último ejemplo, la dimensión devuelta es (2,), puesto que solo tiene una dimensión.

<a name='2'></a>
### Breve repaso de conceptos del álgebra
<a name='2-1'></a>
##### Producto escalar
El producto escalar de dos vectores equivale al producto del traspuesto del primero por el segundo, teniendo en cuenta que el producto se realiza elemento por elemento. Esto es útil para comprender la multiplicación matricial.

$$z = \vec{a}·\vec{w}$$  equivale a $$z = \vec{a}^T * \vec{w}$$

Ejemplo:

In [6]:
a = np.array([1,2])
w = np.array([3,4])

print(f'Productor escalar: {np.dot(a,w)}')
print(f'Multiplicación traspuesta del primero por el segundo: {np.sum(a.T*w)}')

Productor escalar: 11
Multiplicación traspuesta del primero por el segundo: 11


<a name='2-1-1'></a>
##### np.dot()

`numpy.dot(a, b, out=None)`

Producto escalar de dos matrices. Específicamente,

- Si a y b son matrices 1-D, es el producto interior de vectores (sin conjugación compleja).

- Si a y b son matrices bidimensionales, es una multiplicación de matrices, pero es preferible utilizar `np.matmul` o `a @ b`.

Fuente: https://numpy.org/doc/stable/reference/generated/numpy.dot.html

<a name='2-2'></a>
#### Multiplicación matricial

Recordemos que la multiplicación matricial sigue la regla: **(n,k)\*(k,m) = (n,m)**. Es decir, la primera matriz tiene que tener el mismo número de columnas que número de filas tiene la segunda.

Para la **multiplicación de un vector 1D por una matriz**, el vector debe tener el mismo número de elementos que filas la matriz (k,m). En caso de ser un vector fila, es decir, (k,1), se coge el vector traspuesto:

$$ Z = \vec{a}^T * W = [\vec{a}^T*w_1, ..., \vec{a}^T*w_m, \vec{a}^T*w_m] $$

siendo $w_i$ cada vector columna que compone la matriz W y _m_ el número de columnas de la matriz.

En el **caso general** de multiplicación matriz por matriz, se multiplica cada fila de la primera por cada columna de la segunda. Es decir, si consideramos la matriz _A_ como un conjunto de vectores fila y la matriz _W_ como un conjunto de vectores columna:

$$ A = (\vec{F_1}, \vec{F_2}, ..., \vec{F_n)} $$
$$ W = (\vec{C_1}, \vec{C_2}, ..., \vec{C_m)} $$

el producto de estas matrices será:

$$ A*W = 
\begin{pmatrix}
\vec{F_1}*\vec{C_1} & \vec{F_1}*\vec{C_2} & ... & \vec{F_1}*\vec{C_m} \\
\vec{F_2}*\vec{C_1} & \vec{F_2}*\vec{C_2} & ... & \vec{F_2}*\vec{C_m} \\
... & ... & ... & ...\\
\vec{F_n}*\vec{C_1} & \vec{F_n}*\vec{C_2} & ... & \vec{F_n}*\vec{C_m}
\end{pmatrix} $$

Hay un **matiz importante** en el aprendizaje automático. Como se ve en este trabajo, los valores, datos, etc se colocan/apilan en las matrices como vectores columna. Entonces, para que el resultado de los cálculos de la red neuronal sean correctos (por ejemplo, imaginemos que estamos multiplicando la matriz de datos de entrada por la matriz de pesos), hay que convertir la primera matriz en una matriz de vectores fila. Para eso se hace la traspuesta con `.T` y ya se podría realizar el cálculo. En las dos siguientes figuras se muestra un ejemplo:

<img src="imagenes/matriz_traspuesta.png" style="width:110px;height:80px;">
<caption><center><b>Figura 1</b></center></caption><br>

<img src="imagenes/multiplicacion_traspuesta.png" style="width:280px;height:80px;">
<caption><center><b>Figura 2</b></center></caption><br>

<img src="imagenes/resultado.png" style="width:140px;height:70px;">
<caption><center><b>Figura 3</b></center></caption><br>

Veamos cómo aplicarla con Python y NumPy. Para más información y casos, consultar la documentación.
<a name='2-2-1'></a>
##### np.matmul()

`numpy.matmul(x1, x2, /, out=None, ...)`

Producto matricial de dos matrices.

- Si ambos argumentos son bidimensionales, se multiplican como las matrices convencionales.

- Si el primer argumento es 1-D, se convierte en una matriz añadiendo un 1 a sus dimensiones. Tras la multiplicación matricial, se elimina el 1 antepuesto.

- Si el segundo argumento es 1-D, se convierte en una matriz añadiendo un 1 a sus dimensiones. Tras la multiplicación matricial, se elimina el 1 añadido.

.matmul difiere de .dot en dos aspectos importantes:

- No se permite la multiplicación por escalares, en su lugar utilice *.

- Las pilas de matrices se emiten juntas como si las matrices fueran elementos, respetando la regla (n,k)x(k,m)->(n,m):

La función matmul implementa la semántica del operador @ introducido en Python 3.5.

Fuente: https://numpy.org/doc/stable/reference/generated/numpy.matmul.html#numpy.matmul

<a name='2-2-2'></a>
##### a @ b

El operador @ puede utilizarse como abreviatura de np.matmul() en ndarrays.

In [7]:
# Ejemplo
# Matriz 2x3
A = np.array([[1,-1,2],
              [2,-2,1]])
# Matriz 3x2
AT = A.T
# Matriz 2x4
W = np.array([[3,5,7,9],
              [4,6,8,10]])
# Multiplicación de matrices
Z1 = np.matmul(AT,W)
Z2 = AT @ W

print(f'Dimensión del producto AT*W: {Z1.shape}')
print(f'¿Es equivalente np.matmul y @?: {np.array_equal(Z1,Z2)}')

Dimensión del producto AT*W: (3, 4)
¿Es equivalente np.matmul y @?: True


<a name='3'></a>
### Vectorización en redes neuronales

La vectorización es, en pocas palabras, el arte de deshacerse de los bucles `for` explícitos en el código. En la era del aprendizaje profundo, especialmente en la práctica, a menudo los conjuntos de datos de entrenamiento son relativamente grandes, dado que es cuando los algoritmos de aprendizaje profundo tienden a destacar. Por lo tanto, es importante que el código se ejecute los más rápidamente posible porque, de lo contrario, si se está entrenando un gran conjunto de datos, el código puede tardar mucho tiempo en ejecutarse y obtener el resultado. Así que en la era del aprendizaje profundo, la capacidad de realizar la vectorización se ha convertido en una habilidad clave.

<a name='3-0'></a>
#### Cálculo de z

Recordemos que "z" era la combinación lineal de los pesos por las características de cada ejemplo, más un valor umbral: $z = w^Tx + b$. Dados los vectores columna $w \in \mathbb{R}^n$ y $x \in \mathbb{R}^n$, veamos la forma vectorizada y no vectorizada a la hora de calcular "z".

In [8]:
import time

n = 1000000
w = np.random.rand(1000000)
x = np.random.rand(1000000)
b = 7
z = 0

t_ini = time.time()
# Ejemplo implementacion NO vectorizada
for i in range(n):
    z += w[i]*x[i]
z += b
t_fin = time.time()
print(z)
t_no_vec = 1000*(t_fin - t_ini)
print(f'Versión no vectorizada (for loop): {t_no_vec} ms')

# Ejemplo implementacion vectorizada
z = 0
t_ini = time.time()
z = np.dot(w,x) + b
t_fin = time.time()
t_vec = 1000*(t_fin - t_ini)
print(z)
print(f'Versión vectorizada: {t_vec} ms')

print(f'La versión vectorizada es {round(t_no_vec/t_vec)} veces más rápida que la versión NO vectorizada.')

250021.19221575942
Versión no vectorizada (for loop): 332.0481777191162 ms
250021.19221576414
Versión vectorizada: 0.9739398956298828 ms
La versión vectorizada es 341 veces más rápida que la versión NO vectorizada.


No solo la implementación vectorizada es computacionalmente más rápida, sino que además necesita de menos líneas de código.

<a name='3-1'></a>
#### Propragación hacia delante / _forward propagation_

He aquí un ejemplo de cómo se podría implementar la propagación hacia delante (_forward propagation_) en una sola capa. _x_  es la entrada, _W_, los pesos de la primera, segunda y tercera neuronas y los parámetros de sesgo, _b._ Con los valores numéricos elegidos, obtendremos como resultado tres números.

In [9]:
# Las matrices por convención se escriben en mayúscula y los vectores en minúscula.
x = np.array([200, 17])     # Vector 1D
W = np.array([[1, -3, 5],
              [-2, 4, -6]]) # Matriz 2x3
b = np.array([-1, 1, 2])    # Vector 1D

In [10]:
# Vamos a utilizar en el ejemplo una función de activación sigmoide
# Para más detalles sobre su funcionamiento, consultar anexo de funciones NumPy
def sigmoide(z):
    """
    Calcula la función sigmoide de z

    Parámetros
    ----------
    z : Un escalar o matriz numpy de cualquier tamaño.

    Devuelve
    -------
     g : sigmoid(z)
    """
    z = np.clip( z, -500, 500 )     # protección contra el desbordamiento
    g = np.round(1.0/(1.0+np.exp(-z)))

    return g

In [11]:
# Versión NO VECTORIZADA
def forward_propagation(a_entrada, W, B, g):
    """
    Calcula las activaciones de una capa.
    
    Argumentos:
    ----------
      a_entrada (ndarray (n, ))  : Datos de entrada 
      W         (ndarray (n,j))  : Matriz de pesos, n características por unidad, j unidades
      b         (ndarray (j, ))  : vector de sesgo, j unidades
      g                          : función de activación
      
    Devuelve
    ----------
      a_salida  (ndarray (j,))   : j unidades
    """
    unidades = W.shape[1]
    a_salida = np.zeros(unidades)
    for j in range(unidades):               
        w = W[:,j]                                    
        z = np.dot(a_entrada, w) + b[j]
        # Función de activación g() definida fuera de la función
        a_salida[j] = g(z)               
    return(a_salida)

In [12]:
# %%time - Se podría utilizar para ver la diferencia de tiempos de ejecución, pero esta diferencia se nota con entradas numéricas grandes.

a_salida = forward_propagation(x, W, b, sigmoide)
print(a_salida)
print(a_salida.shape)

[1. 0. 1.]
(3,)


¿Qué sucede en la línea `np.dot(w, a_entrada)`? La multiplicación es entre vectores 1D, puesto que `W[:,j]` al indexar una matriz con : en la primera dimensión y 0 en la segunda dimensión, se seleccionan todos los elementos de la primera fila y el primer elemento de cada columna. Por lo tanto, np.dot() está calculando el producto interno de dos vectores 1D y no está realizando una multiplicación matricial.

In [13]:
print(np.dot(W[:,0],x))
print(W[:,0])
print(W[:,0].shape)

166
[ 1 -2]
(2,)


Implementemos a continuación la versión vectorizada. Para ello trabajaremos solo con **matrices**. Además, utilizando `np.matmul()`o su equivalente, `@` reducimos también líneas de código (son las formas en las que NumPy realiza la multiplicación de matrices).

In [14]:
# Redefinimos los valores.
X_matriz = np.array([[200, 17]])     # Matriz 1x2
W_matriz = np.array([[1, -3, 5],
                     [-2, 4, -6]])   # Matriz 2x3
B_matriz = np.array([[-1, 1, 2]])    # Matriz 1x3

**¡Importante!** Por lo visto en el breve repaso de la multiplicación de matrices, dependiendo si multiplicamos $X*W$ o $W*X$, la primera matriz tendrá que ser su traspuesta. Es decir, que siendo todas ellas matrices columnas, en el primer caso sería $X^T*W$y en el segundo $W^T*X$.

In [15]:
# Versión VECTORIZADA
def forward_propagation_vectorizada(A_entrada, W, B, g):
    """
    Calcula las activaciones de una capa.
    
    Argumentos:
    ----------
      A_entrada (ndarray (1,n))  : Matriz de datos de entrada 
      W         (ndarray (n,j))  : Matriz de pesos, n características por unidad, j unidades
      B         (ndarray (1,j))  : Matriz de sesgo, j unidades
      g                          : Función de activación
      
    Devuelve
    ----------
      A_salida  (ndarray (j,))   : j unidades
    """                               
    Z = np.matmul(A_entrada, W) + B # equivalente a: A_entrada @ W + B
    # Función de activación g() definida fuera de la función
    A_salida = g(Z)               
    return(A_salida)

In [16]:
a_salida_vectorizada = forward_propagation_vectorizada(X_matriz, W_matriz, B_matriz, sigmoide)
print(a_salida_vectorizada)
print(a_salida_vectorizada.shape)

[[1. 0. 1.]]
(1, 3)


Como no podía ser de otra manera, obtenemos el mismo resultado numérico. Eso sí, fijémonos que `a_salida` es una array 1D de dimensiones (3,), mientras que `a_salida_vectorizada` es una matriz (o vector 2D) de dimensiones (1x3).

Esta resulta ser una implementación muy eficiente de un paso de propagación hacia delante a través de una capa en una red neuronal.

<a name='3-2'></a>
#### Implementación de las funciones de pérdida L1 y L2

En optimización matemática y teoría de la decisión, una función de pérdida o función de coste (a veces también llamada función de error) es una función que asigna un suceso o valores de una o más variables a un número real que representa intuitivamente algún "coste" asociado al suceso. 

La pérdida se utiliza para evaluar el rendimiento del modelo. Cuanto mayor sea la pérdida, más diferentes serán las predicciones ($ \hat{y} $) de los valores reales ($y$). En el aprendizaje profundo, se utilizan algoritmos de optimización como el descenso del gradiente (_Gradient Descent_) para entrenar el modelo y minimizar el coste.

<a name='3-2-1'></a>
##### L1
La función de pérdida L1, también conocida como función de pérdida por error absoluto, es la diferencia absoluta entre una predicción y el valor real, calculada para cada ejemplo de un conjunto de datos. La agregación de todos estos valores de pérdida se denomina función de coste, donde la función de coste para L1 suele ser MAE (Error Absoluto Medio).

- Implementación de la versión NumPy vectorizada de la pérdida L1. Su definición es:
$$\begin{align*} & L_1(\hat{y}, y) = \sum_{i=0}^{m-1}|y^{(i)} - \hat{y}^{(i)}| \end{align*}\tag{6}$$

In [17]:
def L1(yhat, y):
    """
    Argumentos: 
    yhat    – vector de tamaño m (etiquetas predichas) 
    y       – vector de tamaño m (etiquetas verdaderas) 
    
    Devuelve: 
    perdida – el valor de la función de pérdida L1 definida anteriormente
    """
    perdida = np.sum(abs(y - yhat))
    
    return perdida

In [18]:
# Ejemplo de aplicación
yhat = np.array([.8, 0.2, 0.1, .5, .8])
y = np.array([1, 0, 0, 1, 1])
print("L1 = " + str(L1(yhat, y)))

L1 = 1.2


<a name='3-2-2'></a>
##### L2

La función de pérdida L2, también conocida como función de pérdida por error al cuadrado, es la diferencia al cuadrado entre una predicción y el valor real, calculada para cada ejemplo de un conjunto de datos. La agregación de todos estos valores de pérdida se denomina función de coste, donde la función de coste para L2 suele ser MSE (Mean of Squared Errors).

- Implementación de la versión NumPy vectorizada de la pérdida L2. Hay varias formas de implementar la pérdida L2; una de ellas es utilizando la función np.dot(). Si $\vec{x} = [x_1, x_2, ..., x_m]$, entonces np.dot($\vec{x},\vec{x}$) = $\sum_{i=0}^m x_i^{2}$. 

$$\begin{align*} & L_2(\hat{y},y) = \sum_{i=0}^{m-1}(y^{(i)} - \hat{y}^{(i)})^2 \end{align*}\tag{7}$$

In [19]:
def L2(yhat, y):
    """
    Argumentos: 
    yhat     – vector de tamaño m (etiquetas predichas) 
    y        – vector de tamaño m (etiquetas verdaderas) 
    
    Devuelve: 
    perdida  – el valor de la función de pérdida L2 definida anteriormente
    """
    perdida = np.dot(y - yhat, y - yhat)
    
    return perdida

In [20]:
# Ejemplo de aplicación
yhat = np.array([.8, 0.2, 0.1, .5, .8])
y = np.array([1, 0, 0, 1, 1])
print("L2 = " + str(round(L2(yhat, y),3)))

L2 = 0.38


Otras referencias: https://realpython.com/numpy-array-programming/